## Treating a function like a object

In [1]:
def factorial(n):
    """
    return n!
    """
    return 1 if n<2 else n*factorial(n-1)

In [2]:
factorial(5)

120

In [4]:
print(factorial.__doc__)


    return n!
    


In [6]:
print(type(factorial))

<class 'function'>


In [7]:
fact = factorial 
fact(5)

120

The map function returns an iterable where each item is the result of the application of the first argument (function) to successive elements of the second argument(an iterable)

In [8]:
print(map(fact,range(10)))

In [9]:
print(list(map(fact,range(10))))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]


## Higer-Order Functions

A function that takes a function as argument or returns a function as the result is a high-order function

In [11]:
fruits = ['strawberry','fig','apple','cherry']
sorted(fruits,key=len)

['fig', 'apple', 'cherry', 'strawberry']

In [12]:
def reverse(word):
    return word[::-1]
sorted(fruits,key=reverse) #sorting a list of words by their reversed spelling

['apple', 'fig', 'strawberry', 'cherry']

## User-Defined Callable Types
Not only are Python functions real objects, but arbitrary Python objects may also be made to behave like functions. Implementing a \_\_call\_\_ instance method is all it takes. 

In [18]:
import random 

class BingoCage:
    def __init__(self,items):
        self._items = list(items)
        random.shuffle(self._items)
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        return self.pick()

In [19]:
bingo = BingoCage(range(3))
bingo._items

[0, 1, 2]

In [20]:
bingo()

2

In [21]:
callable(bingo)

True

## From Positional to Keyword-Only Parameters

In [25]:
def tag(name,*content,cls=None,**attrs):
    """Generate one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' {0}={1}'.format(attr,value) 
                           for attr, value 
                           in sorted(attrs.items()))
    else:
        attr_str=''
    if content:
        return '\n'.join('<{0}{1}>{2}</{3}>'.format(name,attr_str,c,name) for c in content)
    else:
        return '<{0}{1} />'.format(name,attr_str)

In [26]:
tag('br')

'<br />'

In [28]:
list1 = ['hello','world']
print(tag('br',*list1))

<br>hello</br>
<br>world</br>


In [29]:
print(tag('br','hello','world'))

<br>hello</br>
<br>world</br>


In [30]:
tag('p','hello',id=33)

'<p id=33>hello</p>'

In [32]:
my_tag = {'name':'img','title':'Sunset Boulevard','Src':'sunset.jpg','cls':'framed'}
print(tag(**my_tag))

<img Src=sunset.jpg class=framed title=Sunset Boulevard />


In [34]:
def clip(text,max_len=80):
    """return text clipped at the last space before or after max_len"""
    end = None
    if len(text) > max_len :
        space_before = text.rfind(' ',0,max_len)
        if space_before >= 0:
            end = space_before 
        else:
            space_after = text.rfind(' ',max_len)
            if space_after >0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()


In [35]:
clip('Hello World',max_len=5)

'Hello'

In [36]:
clip.__code__

<code object clip at 0x0000027A8124E870, file "C:\Users\haha\AppData\Local\Temp\ipykernel_7868\2140354106.py", line 1>

In [37]:
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

In [38]:
clip.__code__.co_argcount

2

In [40]:
from inspect import signature 
sig = signature(clip)
print(str(sig))
for name, param in sig.parameters.items():
    print(param.kind,':',name,'=',param.default)

(text, max_len=80)
POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


The bind method takes any number of arguments and binds them to the parameters in the signature.

In [41]:
sig = signature(tag)
bound_args = sig.bind(**my_tag)
for name, value in bound_args.arguments.items():
    print(name,'=',value)

name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'Src': 'sunset.jpg'}


## Function Annotations

In [47]:
def clip(text:str,max_len:'int>0'=80) -> str:
    """return text clipped at the last space before or after max_len"""
    end = None
    if len(text) > max_len :
        space_before = text.rfind(' ',0,max_len)
        if space_before >= 0:
            end = space_before 
        else:
            space_after = text.rfind(' ',max_len)
            if space_after >0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

In [48]:
print(clip.__annotations__)

{'text': <class 'str'>, 'max_len': 'int>0', 'return': <class 'str'>}


In [49]:
clip('Hello World',max_len=5)

'Hello'

In [52]:
sig = signature(clip)
print(sig.return_annotation)

<class 'str'>


In [53]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note,':',param.name,'=',param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int>0'       : max_len = 80


## Packages for functional Programming

In [54]:
from functools import reduce

def fact(n):
    return reduce(lambda a,b:a*b, range(1,n+1))

In [55]:
fact(5)

120

In [56]:
from operator import mul 
def fact2(n):
    return reduce(mul,range(1,n+1))

In [57]:
fact2(5)

120

itemgetter(1) is equivalent to lambda fields: fields[1]

In [58]:
metro_data = [
    ('Tokyo','JP',36.933,(35.68,139)),
    ('Delhi NCR','IN',21.935,(28.61,77.21))
]

In [59]:
from operator import itemgetter

cc_name = itemgetter(0,1)
for city in metro_data:
    print(cc_name(city))

('Tokyo', 'JP')
('Delhi NCR', 'IN')


In [60]:
from collections import namedtuple
LatLong = namedtuple('LatLong','lat long')
Metropolis = namedtuple('Metropolis','name cc pop coord')
metro_areas = [Metropolis(name,cc,pop,LatLong(lat, long)) for name,cc,pop,(lat,long) in metro_data]

In [61]:
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.68, long=139))

In [62]:
metro_areas[0].coord.long

139

In [66]:
from operator import attrgetter
name_long = attrgetter('name','coord.long')
for city in sorted(metro_areas,key=attrgetter('coord.long')):
    print(name_long(city))

('Delhi NCR', 77.21)
('Tokyo', 139)


In [68]:
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [70]:
str.upper(s)

'THE TIME HAS COME'

In [71]:
hiphenate = methodcaller('replace',' ','-')
hiphenate(s)

'The-time-has-come'

In [72]:
s.replace(' ','-')

'The-time-has-come'

## Freezing Argumets with functools.partial
functools.partial is a higher-order function that allows partial application of a function

In [73]:
from operator import mul
from functools import partial
triple = partial(mul,3) #Create new triple function from mul, binding first positional argument to 3
triple(7)

21